**Transfer Learning with vgg16 - Dog Breed Prediction**

In [1]:
import numpy as np
from glob import glob
import os
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

import torchvision.transforms as transforms
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torchvision.models as models
import torch.nn as nn

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

download data from https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip

**Prepare data**

In [5]:
def Data_Loader(data_path,data_type,batch_size,shuffle=True):
    data_transform = transforms.Compose([
        transforms.Resize(size=(255)),
        transforms.CenterCrop((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225]),
    ])
    
    image_folder = ImageFolder(os.path.join(data_path,data_type),transform = data_transform)
    data_loader = DataLoader(image_folder,batch_size,shuffle)
    return data_loader

In [6]:
data_path = "./data/dogImages/"
train_loader = Data_Loader(data_path ,data_type='train',batch_size=1,shuffle=True)
valid_loader = Data_Loader(data_path,data_type='valid',batch_size=1,shuffle=False)
test_loader = Data_Loader(data_path,data_type='test',batch_size=1,shuffle=False)

**Prepare train and evaluation models**

In [9]:
def train_model(model,optimizer,criterion,data_loader):
    model.train()
    epoch_loss = 0
    epoch_correct = 0
    epoch_total = 0
    for batch_idx, (images, labels) in enumerate(data_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).to(device)
        loss = criterion(outputs,labels).to(device)
        epoch_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        epoch_total += labels.size(0)
        epoch_correct += (predicted == labels).sum().item()
        loss.backward()
        optimizer.step() 
    return epoch_loss/len(data_loader), epoch_correct/epoch_total    

In [10]:
def evaluate_model(model,optimizer,criterion,data_loader):
    model.eval()
    with torch.no_grad():
        epoch_loss = 0
        epoch_correct = 0
        epoch_total = 0
        for batch_idx, (images, labels) in enumerate(data_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).to(device)
            loss = criterion(outputs,labels).to(device)
            epoch_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            epoch_total += labels.size(0)
            epoch_correct += (predicted == labels).sum().item()
    return epoch_loss/len(data_loader), epoch_correct/epoch_total

**Train last layer of vgg16**

In [13]:
# download pretrained vgg16 model from torchvision.models
vgg16_pretrained= models.vgg16(pretrained=True)

# freeze the model weights
for param in vgg16_pretrained.parameters():
    param.requires_grad = False

# change the output dimension of the pretrained model's last layer to 133, the number of dog breeds in our train data
vgg16_pretrained.classifier[6] = nn.Linear(4096, 133)
vgg16_pretrained.to(device)

criterion_pretrained = nn.CrossEntropyLoss().to(device)
optimizer_pretrained = torch.optim.SGD(vgg16_pretrained.classifier[6].parameters(), lr=0.001)
n_epochs = 20
for epoch in range(n_epochs):
    
    train_loss, train_acc = train_model(vgg16_pretrained,optimizer_pretrained,criterion_pretrained,data_loader=train_loader)
    val_loss, val_acc = evaluate_model(vgg16_pretrained,optimizer_pretrained,criterion_pretrained,data_loader=valid_loader)
        
    print(f'Epoch: {epoch+1} Train Loss: {train_loss:.3f} Train Acc: {train_acc*100:.2f}% Val Loss: {val_loss:.3f} Val Acc: {val_acc*100:.2f}%')

Epoch: 1 Train Loss: 1.337 Train Acc: 66.47% Val Loss: 0.604 Val Acc: 82.51%
Epoch: 2 Train Loss: 0.576 Train Acc: 82.84% Val Loss: 0.536 Val Acc: 82.63%
Epoch: 3 Train Loss: 0.476 Train Acc: 85.31% Val Loss: 0.439 Val Acc: 86.95%
Epoch: 4 Train Loss: 0.403 Train Acc: 87.37% Val Loss: 0.417 Val Acc: 86.71%
Epoch: 5 Train Loss: 0.359 Train Acc: 89.09% Val Loss: 0.433 Val Acc: 86.83%
Epoch: 6 Train Loss: 0.327 Train Acc: 89.51% Val Loss: 0.403 Val Acc: 86.83%
Epoch: 7 Train Loss: 0.301 Train Acc: 90.51% Val Loss: 0.397 Val Acc: 87.54%
Epoch: 8 Train Loss: 0.294 Train Acc: 90.54% Val Loss: 0.404 Val Acc: 86.83%
Epoch: 9 Train Loss: 0.282 Train Acc: 91.14% Val Loss: 0.401 Val Acc: 87.43%
Epoch: 10 Train Loss: 0.248 Train Acc: 92.13% Val Loss: 0.416 Val Acc: 87.19%
Epoch: 11 Train Loss: 0.243 Train Acc: 91.87% Val Loss: 0.390 Val Acc: 88.14%
Epoch: 12 Train Loss: 0.236 Train Acc: 92.66% Val Loss: 0.405 Val Acc: 86.95%
Epoch: 13 Train Loss: 0.230 Train Acc: 92.96% Val Loss: 0.410 Val Acc: 87

**Save trained parameters**

In [14]:
PATH = "trained_VGG16Net.pt"
torch.save(vgg16_pretrained.state_dict(),PATH)